## 2. analyse statistiques bivariées

### a. Tableau de contingence

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency # type: ignore


In [7]:
file_path = "../../data/dataset.csv"
df = pd.read_csv(file_path)


In [8]:
def tableau_contingence(df, target='Treatment'):
    """Affiche les tableaux de contingence de toutes les variables avec la variable cible."""
    print("="*80 + f"\nTABLEAUX DE CONTINGENCE - TOUTES LES VARIABLES × {target}\n" + "="*80)
    
    resultats = {}
    for col in df.columns:
        if col == target:
            continue
        
        print(f"\n{'='*80}\nVariable : {col} × {target}\n{'='*80}")
        
        # Effectifs
        contingence = pd.crosstab(df[col], df[target], margins=True, margins_name="TOTAL")
        print("\n--- EFFECTIFS ---")
        print(contingence)
        
        # Proportions en ligne (%) - Distribution de Treatment selon la variable
        contingence_row = pd.crosstab(df[col], df[target], normalize='index') * 100
        print(f"\n--- PROPORTIONS EN LIGNE (%) - Distribution de {target} selon {col} ---")
        print(contingence_row.round(2))
        
        resultats[col] = {'effectifs': contingence, 'pct_ligne': contingence_row}
    
    print("\n" + "="*80 + "\nANALYSE TERMINÉE\n" + "="*80)
    return resultats

# Analyse de toutes les variables avec Treatment
resultats_contingence = tableau_contingence(df)


TABLEAUX DE CONTINGENCE - TOUTES LES VARIABLES × Treatment

Variable : Gender × Treatment

--- EFFECTIFS ---
Treatment      No     Yes   TOTAL
Gender                           
Female       9406   21308   30714
Male       123552  107062  230614
TOTAL      132958  128370  261328

--- PROPORTIONS EN LIGNE (%) - Distribution de Treatment selon Gender ---
Treatment     No    Yes
Gender                 
Female     30.62  69.38
Male       53.58  46.42

Variable : Country × Treatment

--- EFFECTIFS ---
Treatment                   No     Yes   TOTAL
Country                                       
Australia                 1397    2147    3544
Belgium                    477       0     477
Bosnia and Herzegovina     237       0     237
Brazil                     886     452    1338
Canada                    6802    7375   14177
Colombia                   384       0     384
Costa Rica                 384       0     384
Croatia                      0     385     385
Czech Republic             38

- Les femmes aurais une plus grande probabilité d'être sous traitement que les hommes 
- Avoir un antécedent familliale augmenterais les probabilités d'être sous traitements
- Avoir une option d'assurance santé augmenterais les chances d'être sous traitements
Faits intéressants et contre-intuitif : 
    - Allez souvent dehors ou non ne changerais rien à la probabilité d'être sous traitements
    - Avoir des antécedants de santé mentale ne changerais pas les probabilités 
    - Les variations de l'humeur n'entrainerait pas de hausse de probabilité d'être sous traitements
    - Les difficultés à gérer les émotions n'augmentent pas les probabilités d'être sous traitements


### b. Chi^2 et p_value

In [ ]:
def test_chi2(df, target='Treatment'):
    """Calcule Chi² et p-value pour toutes les variables vs la cible."""
    print("="*80 + f"\nTEST DU CHI² - ASSOCIATION AVEC {target}\n" + "="*80)
    
    resultats = []
    for col in df.columns:
        if col == target:
            continue
        
        # Tableau de contingence sans marges
        contingence = pd.crosstab(df[col], df[target])
        
        # Test du Chi²
        chi2, p_value, _, _ = chi2_contingency(contingence)
        chi2 = float(chi2) # type: ignore
        p_value = float(p_value) # type: ignore
        
        # Format p-value
        p_str = "< 0.001" if p_value < 0.001 else round(p_value, 4)
        
        resultats.append({
            'Variable': col,
            'Chi²': round(chi2, 2),
            'p-value': p_str
        })
    
    # Affichage sous forme de DataFrame trié par Chi²
    resultats_df = pd.DataFrame(resultats).sort_values('Chi²', ascending=False)
    print("\n" + resultats_df.to_string(index=False))
    
    print("\n" + "="*80)
    print("Interprétation : p-value < 0.05 = association statistiquement significative")
    print("="*80 + "\nANALYSE TERMINÉE\n" + "="*80)
    
    return resultats_df

# Calcul pour toutes les variables
resultats_chi2 = test_chi2(df)


: 

### c. Cramér's V


In [10]:
def cramer_v(df, target='Treatment'):
    """Calcule Cramér's V pour mesurer la force d'association entre variables."""
    print("="*80 + f"\nCRAMÉR'S V - FORCE D'ASSOCIATION AVEC {target}\n" + "="*80)
    
    resultats = []
    for col in df.columns:
        if col == target:
            continue
        
        # Tableau de contingence
        contingence = pd.crosstab(df[col], df[target])
        
        # Chi² et dimensions
        chi2, _, _, _ = chi2_contingency(contingence)
        n = contingence.sum().sum()
        r, c = contingence.shape
        
        # Cramér's V = sqrt(Chi² / (n * (min(r,c) - 1)))
        v = np.sqrt(chi2 / (n * (min(r, c) - 1)))
        
        # Interprétation
        if v < 0.1:
            force = "Négligeable"
        elif v < 0.3:
            force = "Faible"
        elif v < 0.5:
            force = "Modéré"
        else:
            force = "Fort"
        
        resultats.append({
            'Variable': col,
            "Cramér's V": round(v, 3),
            'Force': force
        })
    
    # Affichage trié par Cramér's V
    resultats_df = pd.DataFrame(resultats).sort_values("Cramér's V", ascending=False)
    print("\n" + resultats_df.to_string(index=False))
    
    print("\n" + "="*80)
    print("Interprétation : 0-0.1 (négligeable), 0.1-0.3 (faible), 0.3-0.5 (modéré), >0.5 (fort)")
    print("="*80 + "\nANALYSE TERMINÉE\n" + "="*80)
    
    return resultats_df

# Calcul pour toutes les variables
resultats_cramer = cramer_v(df)


CRAMÉR'S V - FORCE D'ASSOCIATION AVEC Treatment

             Variable  Cramér's V       Force
        FamilyHistory       0.362      Modéré
          CareOptions       0.292      Faible
              Country       0.259      Faible
               Gender       0.148      Faible
MentalHealthInterview       0.092 Négligeable
         SelfEmployed       0.038 Négligeable
     IncreasingStress       0.008 Négligeable
      CopingStruggles       0.007 Négligeable
           Occupation       0.006 Négligeable
  MentalHealthHistory       0.005 Négligeable
          DaysIndoors       0.004 Négligeable
         HabitsChange       0.002 Négligeable
           MoodSwings       0.002 Négligeable
       SocialWeakness       0.002 Négligeable
         WorkInterest       0.002 Négligeable
     SocialWeakness.1       0.002 Négligeable

Interprétation : 0-0.1 (négligeable), 0.1-0.3 (faible), 0.3-0.5 (modéré), >0.5 (fort)
ANALYSE TERMINÉE


La conclusion de l'analyse descriptive est que le dataset pourrait soit synthétique, soit être biaisé car certains résultats sont contre intuitifs. Notamment MentalHealth history. Les données sont indiquées comme étant "self-reported", les biais et la mauvaise qualité des données peuvent venir de là. L'objectif ici, avec Catboost sera de voir la différence entre Cramér's et le SHAP. Les relations non linéaires capturées pas CatBoost pourrait faire ressortir des variables prédictives.